# Evolving a Lunar Lander with differentiable Genetic Programming

In [2]:
import gymnasium as gym

from genepro.node_impl import *
from genepro.evo import Evolution
from genepro.node_impl import Constant

import torch
import torch.optim as optim

import random
import os
import copy
from collections import namedtuple, deque

import matplotlib.pyplot as plt
from matplotlib import animation

In [3]:
cuda = torch.cuda.is_available()
print(cuda)

True


In [4]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)  
print(device)
a = torch.zeros(4,3)    
a = a.to(device)
# print(torch.cuda.set_device(1) )

cuda:0


In [5]:
!pip install pycuda
import torch
import pycuda.driver as cuda
cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name() # '0' is the id of your GPU

'Quadro P1000'

sets it to first one, aka intel hd stuff

In [6]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use GPU 0
else:
    device = torch.device("cpu")     # Use CPU

# Set the device for tensors
torch.cuda.set_device(device)


In [7]:
# cuda.Device(0).name()
# torch.Device(0).name()
# torch.device("cuda:0")

set it to gpu

In [8]:
# import torch

# # Check if CUDA is available
# if torch.cuda.is_available():
#     device = torch.device(torch.cuda.get_device_name(0))  # Use GPU 0
#     # device = torch.device(cuda.Device(0))
    
# else:
#     device = torch.device("cpu")  # Use CPU

# # Set the device for tensors
# torch.cuda.set_device(device)


In [9]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device_index = 0  # Replace with the index of your GPU
    device = torch.device('cuda:' + str(device_index) if torch.cuda.is_available() else 'cpu')
else:
    device = torch.device('cpu')

# Set the device for tensors
torch.cuda.set_device(device)


In [10]:
import torch

# Check the current device
device = torch.cuda.current_device()
print("Current device:", torch.cuda.get_device_name(device))


Current device: Quadro P1000


In [11]:
env = gym.make("LunarLander-v2", render_mode="rgb_array")

In [12]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    def __iadd__(self, other):
      self.memory += other.memory
      return self 

    def __add__(self, other):
      self.memory = self.memory + other.memory 
      return self

In [13]:
frames = []

In [14]:
def fitness_function_pt(multitree, num_episodes=5, episode_duration=300, render=False, ignore_done=False):
  memory = ReplayMemory(10000)
  rewards = []

  for _ in range(num_episodes):
    # get initial state of the environment
    observation = env.reset()
    observation = observation[0]
    
    for _ in range(episode_duration):
      if render:
        frames.append(env.render())

      input_sample = torch.from_numpy(observation.reshape((1,-1))).float()
      
      # what goes here? TODO
      action = torch.argmax(multitree.get_output_pt(input_sample))
      observation, reward, terminated, truncated, info = env.step(action.item())
      rewards.append(reward)
      output_sample = torch.from_numpy(observation.reshape((1,-1))).float()
      memory.push(input_sample, torch.tensor([[action.item()]]), output_sample, torch.tensor([reward]))
      if (terminated or truncated) and not ignore_done:
        break

  fitness = np.sum(rewards)
  
  return fitness, memory

## Evolution Setup
Here the leaf and internal nodes are defined. Think about the odds of sampling a constant in this default configurations. Also think about any operators that could be useful and add them here. 

Adjust the population size (multiple of 8 if you want to use the standard tournament selection), max generations and max tree size to taste. Be aware that each of these settings can increase the runtime.

Base attempt 1, different nodes

In [16]:
num_features = env.observation_space.shape[0]
leaf_nodes = [Feature(i) for i in range(num_features)]
leaf_nodes = leaf_nodes + [Constant()] # Think about the probability of sampling a coefficient
# Atomic funtions for the baseline solution
internal_nodes_base = [Plus(),Minus(),Times(),Div(),Log(),Abs()]
leaf_nodes_base = leaf_nodes # as defined at the beginning

# Create evolution object
base_evo = Evolution(
    fitness_function_pt, internal_nodes_base, leaf_nodes_base, 4,
    pop_size=64, 
    # max_gens=70, 
    max_gens=20,
    max_tree_size=32, 
    n_jobs=6, verbose=True,
    init_max_depth = 4,
    # max_tree_size : int=64,
    # crossover = [{"fun":subtree_crossover, "rate": 0.5}],
    # crossovers = 0.4,
    # mutations = 0.5,
    # coeff_opts = 0.5,
    # selection = 8,
    )

# Run the evolution
base_evo.evolve()

gen: 1,	best of gen fitness: -525.334,	best of gen size: 19
gen: 2,	best of gen fitness: -390.195,	best of gen size: 19
gen: 3,	best of gen fitness: -369.751,	best of gen size: 19
gen: 4,	best of gen fitness: -331.039,	best of gen size: 32
gen: 5,	best of gen fitness: -315.249,	best of gen size: 19
gen: 6,	best of gen fitness: -260.795,	best of gen size: 32


KeyboardInterrupt: 

redo with more gens

In [ ]:
num_features = env.observation_space.shape[0]
leaf_nodes = [Feature(i) for i in range(num_features)]
leaf_nodes = leaf_nodes + [Constant()] # Think about the probability of sampling a coefficient
# Atomic funtions for the baseline solution
internal_nodes_base = [Plus(),Minus(),Times(),Div(),Log(),Abs()]
leaf_nodes_base = leaf_nodes # as defined at the beginning

# Create evolution object
base_evo = Evolution(
    fitness_function_pt, internal_nodes_base, leaf_nodes_base, 4,
    pop_size=64, 
    # max_gens=70, 
    max_gens=70,
    max_tree_size=32, 
    n_jobs=6, verbose=True,
    init_max_depth = 4,
    # max_tree_size : int=64,
    # crossover = [{"fun":subtree_crossover, "rate": 0.5}],
    # crossovers = 0.4,
    # mutations = 0.5,
    # coeff_opts = 0.5,
    # selection = 8,
    )

# Run the evolution
base_evo.evolve()

Base attempt 2, added log

In [ ]:
num_features = env.observation_space.shape[0]
leaf_nodes = [Feature(i) for i in range(num_features)]
leaf_nodes = leaf_nodes + [Constant()] # Think about the probability of sampling a coefficient
internal_nodes = [Plus(),Minus(),Times(),Div(), Log()] #Add your own operators here

evo = Evolution(
  fitness_function_pt, internal_nodes, leaf_nodes,
  4, # default is 4, number of trees in multitree
  pop_size=64,
  max_gens=20,
  max_tree_size=31,
  n_jobs=6,
  verbose=True)
evo.evolve()

GPU attempt up population

In [ ]:
num_features = env.observation_space.shape[0]
leaf_nodes = [Feature(i) for i in range(num_features)]
leaf_nodes = leaf_nodes + [Constant()] # Think about the probability of sampling a coefficient
internal_nodes = [Plus(),Minus(),Times(),Div(), Log()] #Add your own operators here

evo = Evolution(
  fitness_function_pt, internal_nodes, leaf_nodes,
  4, # default is 4, number of trees in multitree
  pop_size=1024,
  max_gens=3,
  max_tree_size=64,
  n_jobs=6,
  verbose=True)
evo.evolve()

Baseline higher population

In [ ]:
# Atomic funtions for the baseline solution
internal_nodes_base = [Plus(),Minus(),Times(),Div(),Log()]
leaf_nodes_base = leaf_nodes # as defined at the beginning

# Create evolution object
base_evo = Evolution(
    fitness_function_pt, internal_nodes_base, leaf_nodes_base, 4,
    pop_size=200, 
    # max_gens=70, 
    max_gens=20,
    max_tree_size=64, 
    n_jobs=6, verbose=True,
    init_max_depth = 4,
    # max_tree_size : int=64,
    # crossover = [{"fun":subtree_crossover, "rate": 0.5}],
    # crossovers = 0.4,
    # mutations = 0.5,
    # coeff_opts = 0.5,
    # selection = 8,
    )

# Run the evolution
base_evo.evolve()

Baseline with ifThenElse, final from 2022

In [ ]:
# Better set of atomic funtions for this problem
internal_nodes_final = [Plus(),Minus(),Times(),IfThenElse()]
leaf_nodes_final = leaf_nodes_base # do no use the previous actions

# Create evolution object
final_evo = Evolution(
    lander_fitness_function_complete, internal_nodes_final, leaf_nodes_final,4, # Use adapted fitness function that performs mirroring
    pop_size=200, max_gens=70, max_tree_size=64, n_jobs=6, verbose=True,
    mutations  = [{"fun":subtree_mutation, "rate": 0.7}],
    crossovers = [{"fun":sided_subtree_crossover, "rate": 0.4}],  # Use the adapted crossover function
    # mutations  = [{"fun":subtree_mutation, "rate": 0.3}],
    # crossovers = [{"fun":sided_subtree_crossover, "rate": 0.5}],  # Use the adapted crossover function
    # selection  = [{"fun":tournament_selection, "tournament_size":4}])
    selection={"fun":tournament_selection,"kwargs":{"tournament_size":4}})
# Run the evolution
final_evo.evolve()

Default settings

In [ ]:
# Create evolution object
default_evo = Evolution(
     fitness_function_pt, internal_nodes_base, leaf_nodes_base, 4,
    pop_size=256, 
    init_max_depth=4,
    max_tree_size=64,
    # crossovers : list=[{"fun":subtree_crossover, "rate": 0.5}],
    # mutations : list= [{"fun":subtree_mutation, "rate": 0.5}],
    # coeff_opts : list = [{"fun":coeff_mutation, "rate": 0.5}],
    # selection : dict={"fun":tournament_selection,"kwargs":{"tournament_size":8}},
    # termination criteria
    # max_evals : int=None,
    # max_gens : int=100,
    # max_time : int=None,
    # other
    # n_jobs : int=4,
    # verbose : bool=False,
    )
# Run the evolution
default_evo.evolve()

## Evolve
Running this cell will use all the settings above as parameters

In [ ]:
evo.evolve()

# Test

In [ ]:
def get_test_score(tree):
    rewards = []

    for i in range(10):
      # get initial state
      observation = env.reset(seed=i)
      observation = observation[0]

      for _ in range(500):    
        # build up the input sample for GP
        input_sample = torch.from_numpy(observation.reshape((1,-1))).float()
        # get output (squeezing because it is encapsulated in an array)
        output = tree.get_output_pt(input_sample)
        action = action = torch.Tensor.argmax(output)# What goes here?
        observation, reward, terminated, truncated, info = env.step(action.item())
        rewards.append(reward)


        output_sample = torch.from_numpy(observation.reshape((1,-1))).float()
        if (terminated or truncated):
            break

    fitness = np.sum(rewards)
    
    return fitness

best = evo.best_of_gens[-1]

print(best.get_readable_repr())
print(get_test_score(best))

## Make an animation
Here the best evolved individual is selected and one episode is rendered. Make sure to save your lunar landers over time to track progress and make comparisons.

In [ ]:
frames = []

# gist to save gif from https://gist.github.com/botforge/64cbb71780e6208172bbf03cd9293553
def save_frames_as_gif(frames, path='./', filename='evolved_lander.gif'):
  plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
  patch = plt.imshow(frames[0])
  plt.axis('off')
  def animate(i):
      patch.set_data(frames[i])
  anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
  anim.save(path + filename, writer='imagemagick', fps=60)

frames = []
fitness_function_pt(best, num_episodes=1, episode_duration=500, render=True, ignore_done=False)
env.close()
save_frames_as_gif(frames)

## Play animation

<img src="evolved_lander.gif" width="750">

## Optimisation
The coefficients in the multi-tree aren't optimised. Here Q-learning (taken from https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) is used to optimise the weights further. Incorporate coefficient optimisation in training your agent(s). Coefficient Optimisation can be expensive. Think about how often you want to optimise, when, which individuals etc.

In [ ]:
batch_size = 128
GAMMA = 0.99

constants = best.get_subtrees_consts()

if len(constants)>0:
  optimizer = optim.AdamW(constants, lr=1e-3, amsgrad=True)

for _ in range(500):

  if len(constants)>0 and len(evo.memory)>batch_size:
    target_tree = copy.deepcopy(best)

    transitions = evo.memory.sample(batch_size)
    batch = Transition(*zip(*transitions))
    
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                        batch.next_state)), dtype=torch.bool)

    non_final_next_states = torch.cat([s for s in batch.next_state
                                               if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = best.get_output_pt(state_batch).gather(1, action_batch)
    next_state_values = torch.zeros(batch_size, dtype=torch.float)
    with torch.no_grad():
      next_state_values[non_final_mask] = target_tree.get_output_pt(non_final_next_states).max(1)[0].float()

    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
   
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(constants, 100)
    optimizer.step()

print(best.get_readable_repr())
print(get_test_score(best))

In [ ]:
frames = []
fitness_function_pt(best, num_episodes=1, episode_duration=500, render=True, ignore_done=False)
env.close()
save_frames_as_gif(frames, filename='evolved_lander_RL.gif')

<img src="evolved_lander_RL.gif" width="750">